## ERM with ResNet18 - No-Adaption Baseline

#### Source Domains: PACS Art/Painting, Cartoon, Photo
#### Target Domain: PACS Sketch

In [1]:
import torch
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader, ConcatDataset
from torch.optim.lr_scheduler import LambdaLR
import random
import numpy as np
import os

device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"

In [2]:
def fix_random_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Fixed random seed: {seed}")

fix_random_seed(42)

# For deterministic DataLoader behavior
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

Fixed random seed: 42


In [3]:
model = torchvision.models.resnet18(weights='IMAGENET1K_V1')

for param in model.parameters():
    param.requires_grad = False
model.fc = torch.nn.Linear(model.fc.in_features, 7)

base_lr = 0.01
weight_decay = 1e-4
warmup_epochs = 3
num_epochs = 30
optim = torch.optim.SGD(
        model.parameters(),
        lr=base_lr,
        momentum=0.9,
        weight_decay=weight_decay
    )

def lr_lambda(current_epoch):
    if current_epoch < warmup_epochs:
        # Linear warmup from 0 -> 1
        return float(current_epoch + 1) / float(max(1, warmup_epochs))
    else:
        # Cosine annealing from 1 -> 0
        progress = (current_epoch - warmup_epochs) / float(max(1, num_epochs - warmup_epochs))
        return 0.5 * (1.0 + torch.cos(torch.tensor(progress * 3.1415926535)))

scheduler = LambdaLR(optim, lr_lambda=lr_lambda)

criterion = torch.nn.CrossEntropyLoss()

In [4]:
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(imagenet_mean, imagenet_std)
])

art_dir = "../data/pacs_data/pacs_data/art_painting"
cartoon_dir = "../data/pacs_data/pacs_data/cartoon"
photo_dir = "../data/pacs_data/pacs_data/photo"
sketch_dir = "../data/pacs_data/pacs_data/sketch"

art_dataset = datasets.ImageFolder(root=art_dir, transform=transform)
cartoon_dataset = datasets.ImageFolder(root=cartoon_dir, transform=transform)
photo_dataset = datasets.ImageFolder(root=photo_dir, transform=transform)
sketch_dataset = datasets.ImageFolder(root=sketch_dir, transform=transform)

g = torch.Generator()
g.manual_seed(42)

art_loader = DataLoader(
    art_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    worker_init_fn=seed_worker,
    generator=g
)

cartoon_loader = DataLoader(
    cartoon_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    worker_init_fn=seed_worker,
    generator=g
)

photo_loader = DataLoader(
    photo_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    worker_init_fn=seed_worker,
    generator=g
)

sketch_loader = DataLoader( # This is also the test domain loader
    sketch_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    worker_init_fn=seed_worker,
    generator=g
)

source_dataset = ConcatDataset([art_dataset, cartoon_dataset, photo_dataset])

source_loader = DataLoader(
    source_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    worker_init_fn=seed_worker,
    generator=g
)

pacs_classes = sketch_dataset.classes

In [5]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.to(device)
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_loss = running_loss / total
    accuracy = 100. * correct / total
    return avg_loss, accuracy

In [6]:
def evaluate(model, test_loader, device):
    model.to(device)
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            preds = model.forward(images)
            preds = torch.argmax(preds, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total * 100
    return accuracy

In [7]:

for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(model, source_loader, criterion, optim, device)
    scheduler.step() 
    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"LR: {scheduler.get_last_lr()[0]:.5f}")

Epoch 1/30 | Loss: 0.6437 | Train Acc: 78.72% | LR: 0.00667
Epoch 2/30 | Loss: 0.3962 | Train Acc: 86.51% | LR: 0.01000
Epoch 3/30 | Loss: 0.3993 | Train Acc: 86.92% | LR: 0.01000
Epoch 4/30 | Loss: 0.3691 | Train Acc: 87.83% | LR: 0.00997
Epoch 5/30 | Loss: 0.3248 | Train Acc: 88.82% | LR: 0.00987
Epoch 6/30 | Loss: 0.3568 | Train Acc: 88.49% | LR: 0.00970
Epoch 7/30 | Loss: 0.2794 | Train Acc: 90.48% | LR: 0.00947
Epoch 8/30 | Loss: 0.2874 | Train Acc: 90.53% | LR: 0.00918
Epoch 9/30 | Loss: 0.2776 | Train Acc: 91.06% | LR: 0.00883
Epoch 10/30 | Loss: 0.2645 | Train Acc: 90.91% | LR: 0.00843
Epoch 11/30 | Loss: 0.2702 | Train Acc: 90.58% | LR: 0.00799
Epoch 12/30 | Loss: 0.2434 | Train Acc: 91.55% | LR: 0.00750
Epoch 13/30 | Loss: 0.2492 | Train Acc: 91.49% | LR: 0.00698
Epoch 14/30 | Loss: 0.2383 | Train Acc: 92.00% | LR: 0.00643
Epoch 15/30 | Loss: 0.2194 | Train Acc: 92.56% | LR: 0.00587
Epoch 16/30 | Loss: 0.2347 | Train Acc: 92.21% | LR: 0.00529
Epoch 17/30 | Loss: 0.2065 | Trai

### Evaluation on Source Domains

In [8]:
art_accuracy = evaluate(model, art_loader, device)
print(f"Art Accuracy: {art_accuracy:.2f}%")

cartoon_accuracy = evaluate(model, cartoon_loader, device)
print(f"Cartoon Accuracy: {cartoon_accuracy:.2f}%")

photo_accuracy = evaluate(model, photo_loader, device)
print(f"Photo Accuracy: {photo_accuracy:.2f}%")

source_accuracy = evaluate(model, source_loader, device)
print(f"\nAll Source Domains Accuracy: {source_accuracy:.2f}%")

Art Accuracy: 95.70%
Cartoon Accuracy: 95.90%
Photo Accuracy: 99.34%

All Source Domains Accuracy: 96.78%


### Evaluation on Test Domain

In [9]:
sketch_accuracy = evaluate(model, sketch_loader, device)
print(f"Sketch Accuracy: {sketch_accuracy:.2f}%")

Sketch Accuracy: 42.73%
